# HW3
Boscato Marco - 2096921

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import data

# Constants
planets_data = data.all_data

# planets standard gravitational parameter
mu = np.array([2.2032e13, 3.24859e14, 3.986e14, 4.282837e13, 1.26686534e17, 3.7931187e16, 5.793939e15, 6.836529e15])


## Exercise 2:
The angles $\Omega$, $i$, $\omega$ are the Euler angles (corresponding to α, β, γ in the notation used in that article) characterizing the orientation of the coordinate system. We want to convert in a cartesian coordinate system, to do so we set $\hat{x}$, $\hat{y}$, $\hat{z}$ from the inertial coordinate frame $\hat{I}$, $\hat{J}$, $\hat{K}$ where $\hat{I}$, $\hat{J}$ is in the equatorial plane of the central body. $\hat{I}$ is in the direction of the vernal equinox. $\hat{J}$ is perpendicular to $\hat{I}$ and with $\hat{I}$ defines the reference plane. $\hat{K}$ is perpendicular to the reference plane. In the Solar System usually the ecliptic as that plane.

$\hat{x}$, $\hat{y}$ are in the orbital plane and with $\hat{x}$ in the direction to the pericenter (periapsis). $\hat{z}$ is perpendicular to the plane of the orbit. $\hat{y}$ is mutually perpendicular to $\hat{x}$ and $\hat{z}$.

Then, the transformation from the $\hat{I}$, $\hat{J}$, $\hat{K}$ coordinate frame to the $\hat{x}$, $\hat{y}$, $\hat{z}$ frame with the Euler angles $\Omega$, $i$, $\omega$ is:

$$
x_1 = \cos \Omega \cdot \cos \omega - \sin \Omega \cdot \cos i \cdot \sin \omega \\
x_2 = \sin \Omega \cdot \cos \omega + \cos \Omega \cdot \cos i \cdot \sin \omega \\
x_3 = \sin i \cdot \sin \omega \\
$$

$$
y_1 = - \cos \Omega \cdot \sin \omega - \sin \Omega \cdot \cos i \cdot \cos \omega \\
y_2 = - \sin \Omega \cdot \sin \omega + \cos \Omega \cdot \cos i \cdot \cos \omega \\
y_3 = \sin i \cdot \cos \omega \\
$$

$$
z_1 = \sin i \cdot \sin \Omega \\
z_2 = - \sin i \cdot \cos \Omega \\
z_3 = \cos i
$$

so we can proced by writing the equation as products of matricies $3 \times 3$, this gives us the rotation matrix from a inertial reference system to Laplace system

In [3]:
def rotation_matrix(Omega, w, i):
    R = np.array([[np.cos(Omega)*np.cos(w) - np.sin(Omega)*np.sin(w)*np.cos(i),
                    np.sin(Omega)*np.cos(w) + np.cos(Omega)*np.sin(w)*np.cos(i),
                    np.sin(w)*np.sin(i)],
                    [-np.cos(Omega)*np.sin(w) - np.sin(Omega)*np.cos(w)*np.cos(i),
                     -np.sin(Omega)*np.sin(w) + np.cos(Omega)*np.cos(w)*np.cos(i),
                     np.cos(w)*np.sin(i)],
                     [np.sin(Omega)*np.sin(i), -np.cos(Omega)*np.sin(i), np.cos(i)]])
    return R

the rotation matrix that transform inversily from Laplace to in ertial is the direct transposition of the matrix calculated above, basically

$$
r_{[L]} = R(\omega, i, \Omega) r_{[I]} \\
r_{[I]} = R^T (\omega, i, \Omega) r_{[L]}
$$

explicity it will be:

$$
x = a[(\cos \Omega \cos \omega - \sin \Omega \sin \omega \cos i)(\cos E - e) - (\cos \Omega \sin \omega + \sin \Omega \cos w \cos i)\sqrt{1 -e^2} \sin E] \\
y = a[(\sin \Omega \cos \omega + \cos \Omega \sin \omega \cos i)(\cos E - e) - (\sin \Omega \sin \omega - \cos \Omega \cos w \cos i)\sqrt{1 -e^2} \sin E] \\
z = a[\sin \omega \sin i (\cos E - e) + \cos \omega \sin i \sqrt{1 - e^2} \sin E]
$$

and the velocity transformation is obtained by deriving with respect to time:

$$
\dot{x} = - \frac{na^2}{r}[(\cos \Omega \cos \omega - \sin \Omega \sin \omega \cos i)\sin E + (\cos \Omega \sin \omega + \sin \Omega \cos w \cos i)\sqrt{1 -e^2} \cos E] \\
\dot{y} = - \frac{na^2}{r}[(\sin \Omega \cos \omega + \cos \Omega \sin \omega \cos i)\sin E + (\sin \Omega \sin \omega - \cos \Omega \cos w \cos i)\sqrt{1 -e^2} \cos E] \\
\dot{z} = - \frac{na^2}{r}[\sin \omega \sin i \sin E - \sqrt{1 - e^2} \cos \omega \sin i \cos E]
$$

to calculate the *eccentric anomaly E* we use a Newton-Raphson method given

$$
M = L - \pi \\
0 = E - e\sin E - M
$$

In [4]:
def Newton_Raphson(f, df, x, tol, iterations):
    while abs(f(x)) > tol and iterations < 1000:
        x = x - f(x)/df(x)
        iterations += 1
    return x

In [5]:
def laplace_to_inertial(params, u):
    L, a, e, i, Omega, w = params[0], params[1], params[2], params[3], params[4], params[5]
    
    # Mean Anomaly M
    M = L - w  
    n = np.sqrt(u/a**3.)  # mean motion
    # Eccentric anomaly (E) by Newton-Raphson
    f = lambda E: E - e*np.sin(E) - M
    df = lambda E: 1 - e*np.cos(E)
    E0 = M      # initial guess
    accuracy = 1e-8
    iter = 0
    E = Newton_Raphson(f, df, E0, accuracy, iter)

    # position 
    x = a*((np.cos(Omega)*np.cos(w) - np.sin(Omega)*np.sin(w)*np.cos(i))*(np.cos(E) - e) - (np.cos(Omega)*np.sin(w) + np.sin(Omega)*np.cos(w)*np.cos(i))*np.sqrt(1-e**2.)*np.sin(E))
    y = a*((np.sin(Omega)*np.cos(w) + np.cos(Omega)*np.sin(w)*np.cos(i))*(np.cos(E) - e) - (np.sin(Omega)*np.sin(w) - np.cos(Omega)*np.cos(w)*np.cos(i))*np.sqrt(1-e**2.)*np.sin(E))
    z = a*(np.sin(w)*np.sin(i)*(np.cos(E) - e) + np.cos(w)*np.sin(i)*np.sqrt(1-e**2.)*np.sin(E))

    # velocity
    r = np.sqrt(x**2. + y**2. + z**2.)

    vx = -(n*a**2./r)*((np.cos(Omega)*np.cos(w) - np.sin(Omega)*np.sin(w)*np.cos(i))*np.sin(E) + (np.cos(Omega)*np.sin(w) + np.sin(Omega)*np.cos(w)*np.cos(i))*np.sqrt(1-e**2.)*np.cos(E))
    vy = -(n*a**2./r)*((np.sin(Omega)*np.cos(w) + np.cos(Omega)*np.sin(w)*np.cos(i))*np.sin(E) + (np.sin(Omega)*np.sin(w) - np.cos(Omega)*np.cos(w)*np.cos(i))*np.sqrt(1-e**2.)*np.cos(E))
    vz = -(n*a**2./r)*(np.sin(w)*np.sin(i)*np.sin(E) - np.cos(w)*np.sin(i)*np.sqrt(1-e**2.)*np.cos(E))

    return np.array([x,y,z,vx,vy,vz])


## Exercise 3
First of all we have to convert each day at 23h of 2024 in JDE and then calculate *T* for every one of them

$$
T = \frac{JDE - 2451545.0}{36525}
$$

We know from a converter that 01/01/2024 at 23h is 2460311.45833 and we know that the day after at the same hour will increment by 1, so we can immediately have the JD for each day of 2024.
From `data.py` we get all the orbit parameters obtained on Jannuary 1 2000 at 12 h UT so that we can calculate the new orbit parameter by the formula

$$
a_0 + a_1 T + a_2 T^2 + a_3 T^3
$$

where T is the time measured in Julian centuries in 36525 ephemeris days from the epoch J2000.0

In [6]:
def time_polynomial(dat, Time):
    a0, a1, a2, a3 = dat[:,0], dat[:,1], dat[:,2], dat[:,3]
    return a0 + a1*Time + a2*Time**2 + a3*Time**3

In [7]:
JD_start = 2460311.45833
JD_end = 2460311.45833 + 365

JD_days = np.linspace(JD_start, JD_end, 365)
T = np.zeros_like(JD_days)

for i in range(len(JD_days)):
    T[i] = (JD_days[i] - 2451545.0)/36525.


planets_orbits_parameters = np.zeros((planets_data.shape[0], len(T), planets_data.shape[1]))
for p in range(planets_data.shape[0]):
    for i in range(len(T)):
        planets_orbits_parameters[p, i] = time_polynomial(planets_data[p], T[i])


In [9]:
planets_orbits_cartesian = np.zeros_like(planets_orbits_parameters)

for p in range(planets_data.shape[0]):
    for i in range(len(T)):
        planets_orbits_cartesian[p, i] = laplace_to_inertial(planets_orbits_parameters[p, i], mu[p])